In [42]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
training_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=True,)
test_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=False,)

**Question 1.** The previous instruction needs to return `torch.Tensor`, to do this, you will add the transform argument in the FashionMNIST function. You can find different transforms in the torchvision documentation. You can define your own tranforms using Lambda.

In [3]:
training_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=True,
                  transform=transforms.ToTensor())
test_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=False,
                  transform=transforms.ToTensor())

In [4]:
type(training_data[1][0])

torch.Tensor

**Question 2.** Change the transform of the dataset so that it leads to a flattened version of this tensor (its size should be 
[
784
])

In [5]:
transform = transforms.Compose([
    # transforms.Resize,
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.flatten(x)),
])

training_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=True,
                  transform=transform)
test_data = datasets\
    .FashionMNIST(".",
                  download=True,
                  train=False,
                  transform=transform)

In [6]:
training_data[0][0].shape

torch.Size([784])

**Question 3.** Using a sequential container, define a variable model_TD1 containing the
model of TD1.

In [7]:
model_td1 = torch.nn.Sequential(
    torch.nn.Linear(in_features=28**2 , out_features=1),
)

In [18]:
model_td1

Sequential(
  (0): Linear(in_features=784, out_features=1, bias=True)
)

## Training the network

**Question 4.** Implement a function `def trainer(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4)` which trains model
given as an input with the Adam optimizer using the rate as the learning rate, batch_size
as the number of inputs fed a once to the model (use torch.utils.data.DataLoader for that)
and loss_fn as the loss function.

In [27]:
loader = DataLoader(
    training_data,
    batch_size=1
)

In [22]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model_td1.parameters(),
                             lr = 1e-4)

In [ ]:
optimizer.zero_grad()

In [44]:
n_epochs= 10
for epoch in tqdm(range(n_epochs)):
    for X_batch, y_batch in loader:
        
        X_batch = X_batch.to(torch.float32)
        y_batch = y_batch.to(torch.float32)

        y_pred = model_td1(X_batch)

        loss = loss_fn(y_pred, y_batch)
        
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

100%|██████████| 10/10 [07:48<00:00, 46.89s/it]


In [ ]:
def trainer(
    data : torch.Tensor,
    model : torch.nn.Sequential,
    loss_fn : torch.optim,
    n_epochs : int = 10,
    batch_size : int = 1,
    learning_rate : float = 1e-4
):
    loader = DataLoader(
        training_data,
        batch_size=batch_size
    )
    
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr = learning_rate)
    for epoch in tqdm(range(n_epochs)):
        for X_batch, y_batch in loader:
            
            X_batch = X_batch.to(torch.float32)
            y_batch = y_batch.to(torch.float32)

            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch)
            
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()

    return model

## Obtaining the scores of the model

In [56]:
scores = []

with torch.no_grad():
    for X_batch, y_batch in loader:
        y_pred = model_td1(X_batch)
        scores.append(
            ((y_pred - y_batch).abs() < .5).item()
        )

In [66]:
scores = torch.tensor(scores)

In [69]:
scores.sum()/scores.shape[0]

tensor(0.3689)